In [ ]:
#import the necessary modules
import os
import numpy as np
import pandas as pd
from subprocess import check_output

In [ ]:
#specify the directory and filetype
directory = '/home/sarvesh/ML_Github/flowers/'
filetype = '/*.jpg'

#declare a directory object
path = os.path.dirname(directory)

In [ ]:
#use check_output method of the subprocess package to check the folders
folders = list(check_output(["ls", path]).split('\n'))

#remove any unnecessary files
folders.remove('create_flowers_dataset.ipynb')
folders.remove('')

In [ ]:
#create two empty lists, one for images and the other for flower labels 
image_list = []
label_list = []
for folder in folders:
    images = [x for x in os.listdir(directory + folder + '/') if x.endswith(filetype[2:])]
    image_list.append(images)
    
    #assign corresponding flower name as label for images
    labels = [folder] * len(images)
    label_list.append(labels)
    
#flatten both the lists
image_list = sum(image_list, [])
label_list = sum(label_list, [])

In [ ]:
#The sorting approach cannot be followed before because :
#When images are sorted lexicographically, image name with least character is placed first
#However when images are sorted by system, the image name with lesser number of characters goes first
#sort labels w.r.t lexicographic order of images
#label_list = [x for _, x in sorted(zip(image_list, label_list))]

#sort the image_list finally
#image_list = sorted(image_list)

In [ ]:
#combine both resulting numpy arrays
df = pd.DataFrame(data = [image_list, label_list]).T
df.columns = ['Image', 'Flower']
df.head()

In [ ]:
#create a csv file containing the above data
df.to_csv('flowers.csv')

In [ ]:
#import the necessary modules for image processing
import cv2

In [ ]:
for folder in folders:  